In [2]:
import pandas as pd
import numpy as np

Read the data into six separate dataframes.

In [3]:
df1 = pd.read_csv("turnstile_200606.csv")
df2 = pd.read_csv("turnstile_200530.csv")
df3 = pd.read_csv("turnstile_200411.csv")
df4 = pd.read_csv("turnstile_200321.csv")
df5 = pd.read_csv("turnstile_200208.csv")
df6 = pd.read_csv("turnstile_200118.csv")

Concatenate the data into one dataframe.

In [4]:
df = pd.concat([df1, df2, df3, df4, df5, df6])

In [5]:
df.head()
df.rename(columns={"EXITS                                                               " : "EXITS"}, inplace=True)

The DATE and TIME columns are object datatypes, which in Pandas is a string.

In [6]:
print('DATE datatype: {}, TIME datatype: "{}"'.format(df['DATE'].dtypes, df['TIME'].dtypes))

DATE datatype: object, TIME datatype: "object"


In [7]:
df['DATETIME'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'])

After conversion, the datatype for the DATETIME column is a datetime.

In [8]:
df['DATETIME'].dtypes

dtype('<M8[ns]')

In [9]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,00:00:00,REGULAR,7419887,2520505,2020-05-30 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,04:00:00,REGULAR,7419888,2520505,2020-05-30 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,08:00:00,REGULAR,7419895,2520514,2020-05-30 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,12:00:00,REGULAR,7419911,2520531,2020-05-30 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,16:00:00,REGULAR,7419940,2520553,2020-05-30 16:00:00


The total entries for each turnstile are observed using the `groupby` function in Pandas. First, two additional columns related to the day of the week and the month are added to further analyze the data.

In [10]:
df['DAY'] = df['DATETIME'].dt.day
df['MONTH'] = df['DATETIME'].dt.month_name()
df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY,MONTH
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,00:00:00,REGULAR,7419887,2520505,2020-05-30 00:00:00,30,May
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,04:00:00,REGULAR,7419888,2520505,2020-05-30 04:00:00,30,May
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,08:00:00,REGULAR,7419895,2520514,2020-05-30 08:00:00,30,May
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,12:00:00,REGULAR,7419911,2520531,2020-05-30 12:00:00,30,May
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/30/2020,16:00:00,REGULAR,7419940,2520553,2020-05-30 16:00:00,30,May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205616,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/17/2020,04:00:00,REGULAR,5554,420,2020-01-17 04:00:00,17,January
205617,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/17/2020,08:00:00,REGULAR,5554,420,2020-01-17 08:00:00,17,January
205618,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/17/2020,12:00:00,REGULAR,5554,420,2020-01-17 12:00:00,17,January
205619,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/17/2020,16:00:00,REGULAR,5554,420,2020-01-17 16:00:00,17,January


Next, because the data for the column ENTRIES is cumulative, daily turnstile entries are calculated by subtracting minimum entries from maximum entries for each day.

In [21]:
s = df.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DAY', 'MONTH'])['ENTRIES'].max() - df.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DAY', 'MONTH'])['ENTRIES'].min()
df1 = s.to_frame()
df1.rename(columns={'ENTRIES' : 'daily_entries'}, inplace = True)
df1 = df1.sort_values('daily_entries', ascending=False)
df[(df['DAY'] == 27) & (df['MONTH'] == 'May') & (df['STATION'] == '161/YANKEE STAD')]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY,MONTH
75178,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/27/2020,00:22:00,REGULAR,6366865,6358290,2020-05-27 00:22:00,27,May
75179,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/27/2020,04:22:00,REGULAR,6366865,6358294,2020-05-27 04:22:00,27,May
75180,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/27/2020,08:22:00,REGULAR,6366938,6358361,2020-05-27 08:22:00,27,May
75181,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/27/2020,12:22:00,REGULAR,6366979,6358425,2020-05-27 12:22:00,27,May
75182,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/27/2020,16:22:00,REGULAR,1627393866,1627433548,2020-05-27 16:22:00,27,May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171340,R262B,R195,05-00-05,161/YANKEE STAD,4BD,IRT,05/27/2020,04:22:00,REGULAR,250579,534269,2020-05-27 04:22:00,27,May
171341,R262B,R195,05-00-05,161/YANKEE STAD,4BD,IRT,05/27/2020,08:22:00,REGULAR,250579,534269,2020-05-27 08:22:00,27,May
171342,R262B,R195,05-00-05,161/YANKEE STAD,4BD,IRT,05/27/2020,12:22:00,REGULAR,250579,534269,2020-05-27 12:22:00,27,May
171343,R262B,R195,05-00-05,161/YANKEE STAD,4BD,IRT,05/27/2020,16:22:00,REGULAR,250579,534269,2020-05-27 16:22:00,27,May
